### YoloV6 Implementation Notebook

In this comprehensive ipynb, I walkthrough the implementation part of the FAS project by using a public data set of approx. 2,000 images to measure four metrics discussed in our paper, which are: precision score, recall score, mAP, and F-1. Since each version of the Yolo algorithm have different variations, individualized environment set up is crucial to gurentee a reliable result.

To train YoloV6 on selected dataset, I will be taking the following steps:


* Environment setup - Image pre-procssing
* Environment setup - GPU setup
* Environment setup - Custom dataset preparation
* Load custom dataset
* Run MT-YOLOv6 training
* Evaluate MT-YOLOv6 performance
* Visualize MT-YOLOv5 training data
* Run MT-YOLOv6 inference on test images


This dataset was originally referenced and used from [Roboflow Face Recognition](https://universe.roboflow.com/rusmux/face-recognition-yy4n4/browse?queryText=&pageSize=50&startingIndex=0&browseQuery=true), it has been processed and labeled further for the use of this project by Zongrui Liu under `ShangXiaoNEUCS/CS7980_Computer_FAS/label.py/`


#GPU Setup


In [ ]:
# Setting up GPU environment
!nvidia-smi

Thu Mar  2 11:11:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Install Dependencies

In [ ]:
# Download MT-YOLOv6 repository and install requirements
!git clone https://github.com/meituan/YOLOv6
%cd YOLOv6
!pip install -r requirements.txt

Cloning into 'YOLOv6'...
remote: Enumerating objects: 2977, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 2977 (delta 106), reused 161 (delta 87), pack-reused 2777
Receiving objects: 100% (2977/2977), 44.14 MiB | 35.62 MiB/s, done.
Resolving deltas: 100% (1681/1681), done.
/content/YOLOv6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB

# Prepare Custom Dataset

Dataset selected is converted into YOLOv6 format, which requires YOLO TXT annotations, organized directories, and a specific `.yaml` config file.

From the original website, we need to repleace our custom dataset code.




In [ ]:
# REPLACE with your custom code snippet generated above to use your data
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="wzPXuwSLb8wOL61JhGUH")
project = rf.workspace("rusmux").project("face-recognition-yy4n4")
dataset = project.version(1).download("mt-yolov6")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached roboflow-0.2.32-py3-none-any.whl (50 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
  Using cached requests_toolbelt-0.10.1-py2.py3-none-any.whl (54 kB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached wget-3.2-py3-none-any.whl
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9
  Attempting uninstall: cycler
    Found existing installation: cycler 0.11.0
    Uninstalling cycler-0.11.0:
      Successfully uninstalled cycler-0.11.0


loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Face-recognition-1 in mt-yolov6:: 100%|██████████| 5643/5643 [00:01<00:00, 3531.78it/s]


# Custom Training Details

There are a number of ways to fine tune training of YOLOv6, like custom configuration files for fine tuning, multi GPU support, and passing custom training arguments.


### Multi GPU Support

YOLOv6 supports single and multi GPU training.

Single GPU:
```
python tools/train.py --batch 256 --conf configs/yolov6s_finetune.py --data data/data.yaml --device 0
```

Multi GPU:
```
python -m torch.distributed.launch --nproc_per_node 4 tools/train.py --batch 256 --conf configs/yolov6s_finetune.py --data data/data.yaml --device 0,1,2,3
```


# Begin Custom Training

We're ready to start custom training.

NOTE: We will modify two of the YOLOv6 training defaults in our custom training example: `epochs` and `image-size`. We will adjust from 400 to 100 epochs in our example for speed. Similarly, we will adjust image size from 600x600 to 416x416, which is the default size for other YOLO models (and makes comparisons easier as well as training slightly faster).


In [ ]:
# run this cell to begin training
!python tools/train.py --batch 32 --conf configs/yolov6s.py --epochs 100 --img-size 416 --data {dataset.location}/data.yaml --device 0 